In [ ]:
from test import *

In [ ]:
BGM_lists = get_BGM_lists(YOUTUBE_CHANNEL)
print(BGM_lists)

In [ ]:
p = multiprocessing.Pool(8)
start = time.time()
total_lists = []
temp = p.map_async(get_list_items, BGM_lists)
print(temp.get())
p.close()
p.join()
